Importing libraries and loading environment variables from a .env file containing the API Key 

In [16]:
import requests
import tmdbsimple as tmdb
import pandas as pd

from dotenv import load_dotenv
import os

# This loads the environment variables from the .env file
load_dotenv()

# This retrieves the API Key
API_KEY = os.getenv("TMDb_API_KEY")

if not API_KEY:
    raise ValueError("TMDB API Key not found. Make sure it's set in the .env file.")

Establishing variables, requesting the URL, setting up the API key...

In [17]:
# Set the TMDb API Key for tmdbsimple
tmdb.API_KEY = API_KEY
base_URL = "https://api.themoviedb.org/3"
url = f"{base_URL}/person/{6384}/movie_credits?api_key={API_KEY}"

print(f"Requesting URL: {url}")
response = requests.get(url)

Requesting URL: https://api.themoviedb.org/3/person/6384/movie_credits?api_key=93b29bf0b851209878217fccd1d8ca1a


Establishing a dataframe and running a basic check for response status codes

In [18]:
# Checking the response status
if response.status_code == 200: 
    try:
        # Parsing JSON response 
        movie_data = response.json()
        print(f"Request Successful!")
        # Extracting 'cast' information and creating a DataFrame
        movies = movie_data.get('cast', [])
        df = pd.DataFrame(movies)
        # Displaying the first few rows of the DataFrame
        print(df.head())
    except requests.exceptions.JSONDecodeError:
        print("Error: Response could not be decoded as JSON")
else:
    print(f"Error: {response.status_code} - {response.reason}")

Request Successful!
   adult                     backdrop_path         genre_ids    id  \
0  False  /gVw5NVGsGDhEeYUS1DeFWeLdQ16.jpg       [18, 10749]   859   
1  False  /stSaFz5CtMXwIj3r4MDjqxqWRot.jpg              [18]   468   
2  False  /3PQLohxgvjUQgF94sY5SEgVKSS6.jpg      [28, 53, 80]  1089   
3  False  /tIG8FwHRiHLOlWJdizp2oxdkSL2.jpg      [14, 28, 27]   561   
4  False  /h5h2zpha5480Nea2saKTyF8KNBk.jpg  [28, 80, 18, 53]  1266   

  original_language        original_title  \
0                en    Dangerous Liaisons   
1                en  My Own Private Idaho   
2                en           Point Break   
3                en           Constantine   
4                en          Street Kings   

                                            overview  popularity  \
0  In 18th century France, Marquise de Merteuil a...      19.824   
1  In this loose adaptation of Shakespeare's "Hen...      18.235   
2  In Los Angeles, a gang of bank robbers who cal...      40.398   
3  John Constant

Taking the initial data I'm getting and structuring it with a DataFrame

In [19]:
columns = ['title', 'release_date', 'character', 'revenue']
refined_movies = []

for movie in movies:
    refined_movies.append({
        'title': movie.get('title'),
        'release_date': movie.get('release_date'),
        'character': movie.get('character'),
        'revenue': movie.get('revenue')
    })
    
df_refined = pd.DataFrame(refined_movies, columns=columns)
print(df_refined[:5])

                  title release_date                     character revenue
0    Dangerous Liaisons   1988-12-21  Le Chevalier Raphael Danceny    None
1  My Own Private Idaho   1991-02-01                   Scott Favor    None
2           Point Break   1991-07-12                   Johnny Utah    None
3           Constantine   2005-02-08              John Constantine    None
4          Street Kings   2008-04-10          Detective Tom Ludlow    None


Setting the base URL for the new 'Whoa' API

In [22]:
whoa_base_url = "https://whoa.onrender.com/"
# I will adjust this based on API needs
endpoint = "whoas/ordered"
print(response.text)

{"movie":"Babes in Toyland (The Director's Cut)","year":1986,"release_date":"1986-12-19","director":"Clive Donner","character":"Jack Fenton / Jack-be-Nimble","movie_duration":"02:25:13","timestamp":"00:12:58","full_line":"Caref-whoa!","current_whoa_in_movie":1,"total_whoas_in_movie":2,"whoa_grouping":null,"poster":"https://images.ctfassets.net/a6ek464hq2lg/7G8GyvfQAl85EeFptdqhZR/abb15177e511c0f2592fa27153c7ba2f/h1qDDRTq2rf8cubZZHzeUwT5Lf6.jpeg","video":{"1080p":"https://videos.ctfassets.net/a6ek464hq2lg/3kXOsgJOAEkr8UyCnbJeOb/6cafad66c133a1635de0c3614c76e47f/Babes_in_Toyland_1__1080p_.mp4","720p":"https://videos.ctfassets.net/a6ek464hq2lg/8myLimtO9vk8AeKrKXm2v/225a9cda44875b4df07a05eb417a1728/Babes_in_Toyland_1__720p_.mp4","480p":"https://videos.ctfassets.net/a6ek464hq2lg/1slIhRcNxSMAS6mt96BQDq/fdb1b35bbbf003f5ba46eca28cc62f3e/Babes_in_Toyland_1__480p_.mp4","360p":"https://videos.ctfassets.net/a6ek464hq2lg/4YevBqwtVUB2I5ga7LCxn/aabb3cc75c40d1e4aef6370f6fd12352/Babes_in_Toyland_1__360p_

Defining the new API call and validating the response

In [21]:
response = requests.get(f"{whoa_base_url}{endpoint}")
if response.status_code == 200:
    try:
        whoa_data = response.json()
        print("Data Retrieved Successfully")
        print(f"Number of movies retrieved: {len(whoa_data)}")
        whoa_movies = []
        for item in whoa_data:
            whoa_movies.append({
                'movie': item.get('movie'),
                'year': item.get('year'),
                'release_date': item.get('release_date'),
                'character': item.get('character'),
                'total_whoas_in_movie': item.get("total_whoas_in_movie")
            })
                
            columns = ['movie', 'year', 'release_date', 'character', 'total_whoas_in_movie']
            df_whoa = pd.DataFrame(whoa_movies, columns=columns)
            print(df_whoa.head())
            print(f"Shape of DataFrame: {df_whoa.shape}")
        
    except requests.exceptions.JSONDecodeError:
        print("Error decoding JSON response")

else:
    print(f"Error: {response.status_code} - {response.reason}")

Data Retrieved Successfully
Number of movies retrieved: 14


AttributeError: 'str' object has no attribute 'get'